In [3]:
import tensorflow as tf
import scipy as sp
from scipy import ndimage
import numpy as np
import os
import sklearn
import imageio
from matplotlib import pyplot as plt
%matplotlib inline

In [17]:
# The inputs and outptus are the maps downloaded from https://www.nnvl.noaa.gov/view/globaldata.html
# The inputs to the model are Land Surface Temperature, Moisture and Longwave Energy. 
# The output is the average precipitation map. 
# All maps are in the average weekly format. The idea is to find correlation between the input maps and 
# the output. 

input_labels = ['fire', 'frac', 'land', 'moisture', 'ndvi', 'longwave_energy'];
files = ['./fire_data.txt', './frac_data.txt', './land_data.txt', './mois_data.txt', './ndvi_data.txt', './olwr_data.txt']
input_dict = {}

#xdim = 256
xdim = 168
ydim = 512
in_chan = 6
out_chan = 1

# Process the input : input_metrics will contain a 2048 x 4096 image with 9 channels (corresponding to 3 inputs)

for label, path in zip(input_labels, files):
    
    input_dict[label] = [];
    input_dict[label] = (np.loadtxt(path,delimiter=',').reshape(-1,xdim,ydim,1))
    print(label, path,input_dict[label].shape)

input_metrics = input_dict[input_labels[0]];
for label in (input_labels[1:]):
    input_metrics = np.concatenate((input_metrics, input_dict[label]), axis=3);

# Process the output : output_metric will contain a 256 x 4096 image with 3 channels
path = './rain_data.txt'
output_metric = (np.loadtxt(path, delimiter=',').reshape(-1,xdim,ydim,1))
output_metric = np.array(output_metric);

print(input_metrics.shape)
print(output_metric.shape)


fire ./fire_data.txt (89, 168, 512, 1)
frac ./frac_data.txt (89, 168, 512, 1)
land ./land_data.txt (89, 168, 512, 1)
moisture ./mois_data.txt (89, 168, 512, 1)
ndvi ./ndvi_data.txt (89, 168, 512, 1)
longwave_energy ./olwr_data.txt (89, 168, 512, 1)
(89, 168, 512, 6)
(89, 168, 512, 1)


In [23]:
# Train-test split
N_images = -5;

input_metrics_test = input_metrics[N_images:,:,:,:];
output_metric_test = output_metric[N_images:,:,:,:];

input_metrics = input_metrics[:N_images,:,:,:];
output_metrics = output_metric[:N_images,:,:,:];

samples = input_metrics.shape[0];

In [24]:
# Function to obtain the next batch based on the input size and the batch size
def next_batch(indices, i):
    
    ind0, ind1 = i*batch_size, np.minimum((i+1)*batch_size, samples)
            
    return input_metrics[indices[ind0:ind1], :, :, :], output_metric[indices[ind0:ind1], :, :, :]

In [112]:
# Building a simple CNN model that looks like an auto-encoder. This is the section to change for a new model.

def conv_net(x):
    
    x = tf.reshape(x, shape=[-1, xdim, ydim, in_chan], name='reshape_x');
    x = tf.cast(x, tf.float32) 
    
    l2_reg = tf.contrib.layers.l2_regularizer(5.0);
    # Encoder
    
    # Scale down by 2x2. Out Channels = 16
    conv1 = tf.nn.relu(tf.contrib.layers.conv2d(conv0, 8, [4, 4], stride=2, padding='SAME', 
                                                biases_initializer=tf.zeros_initializer(),
                                                weights_regularizer=l2_reg));
    
    
    # Scale down by 2x2. Out Channels = 8
    conv2 = tf.nn.relu(tf.contrib.layers.conv2d(conv1, 16, [4, 4], stride=2, padding='SAME', 
                                                biases_initializer=tf.zeros_initializer(),
                                                weights_regularizer=l2_reg));
    
    # Scale down by 2x2. Out Channels = 8
    conv3 = tf.nn.relu(tf.contrib.layers.conv2d(conv2, 32, [4, 4], stride=2, padding='SAME', 
                                                biases_initializer=tf.zeros_initializer(),
                                                weights_regularizer=l2_reg));
    
    # Decoder
    # Scale up by 2x2. Out Channels = 16
    conv4 = tf.nn.relu(tf.contrib.layers.conv2d_transpose(conv3, 32, [4, 4], stride=2, padding='SAME',
                                                          biases_initializer=tf.zeros_initializer(),
                                                          weights_regularizer=l2_reg));
    
    # Scale up by 2x2. Out Channels = 32
    conv5 = tf.nn.relu(tf.contrib.layers.conv2d_transpose(conv2, 16, [4, 4], stride=2, padding='SAME',
                                                          biases_initializer=tf.zeros_initializer(),
                                                          weights_regularizer=l2_reg));
    
    # Scale up by 2x2. Out Channels = 1
    conv6 = tf.nn.relu(tf.contrib.layers.conv2d_transpose(conv5, out_chan, [4, 4], stride=2, padding='SAME',
                                                          biases_initializer=tf.zeros_initializer(),
                                                          weights_regularizer=l2_reg));
    
    
    return conv6;

Y = tf.placeholder(tf.float32, shape=(None,xdim,ydim,out_chan));
X = tf.placeholder(tf.float32, shape=[None,xdim,ydim,in_chan]);

optimizer = tf.train.AdamOptimizer(learning_rate=0.001);

out = conv_net(X);
loss_op = tf.reduce_sum(tf.multiply(Y-out,Y-out));
train_op = optimizer.minimize(loss_op);

In [113]:
# Load colorbar for rain
color_bar = imageio.imread('rain_colorbar.png')
color_bar = color_bar[0, :, 0:3].astype(float)

c_n = np.shape(color_bar)[0]

color_bar_data = np.exp(np.linspace(np.log(1), np.log(375), c_n)).reshape((c_n, 1, 1, 1))
color_bar_data = (color_bar_data - 1) / (375 - 1)

# Function for reconstructing rain image based on output from network
def generate_rainfall_images(rain_data, filename):
    ind = np.argmin(abs(color_bar_data - rain_data), axis=0)
    
    rain_image = np.uint8(color_bar[ind, :].reshape((xdim, ydim, 3)).astype(int))
    
    imageio.imwrite(filename, rain_image)

In [ ]:
num_epochs = 300;
batch_size = 8;
num_batches = int(np.ceil(float(samples) / batch_size));
print(num_batches)

loss_arr = [];
test_arr = [];

saver = tf.train.Saver()

with tf.Session() as sess:

    # Run the initializer
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        indices = np.random.permutation(samples);
        
        # Compute the loss across all the batches
        total_loss = 0;
        for i in range(num_batches):
            x_train, y_train = next_batch(indices, i);
            [loss, train] = sess.run([loss_op, train_op], feed_dict={X: x_train, Y: y_train});            
            total_loss += loss;
        
        loss_arr.append(total_loss / num_batches / batch_size)
        
        # Test data
        x_test = input_metrics_test
        y_test = output_metric_test
        
        [test_acc] = sess.run([loss_op], feed_dict={X: x_test, Y: y_test});
        
        test_arr.append(test_acc / x_test.shape[0]);
        print(epoch, total_loss / num_batches / batch_size, test_acc / x_test.shape[0]);
        
    save_path = saver.save(sess, "./model.ckpt")
    
    # Make rain plots from network outputs    

10
0 83529.40625 83076.8125
1 79780.015625 80058.7875
2 76976.1132812 77587.875
3 74689.028125 75522.6125
4 72680.1796875 73595.9375
5 70835.0601562 71700.61875
6 69013.6496094 69959.0875
7 67317.5285156 68218.9375
8 65757.1 66723.79375
9 64283.1074219 65211.09375
10 62866.1464844 63920.6
11 61471.0796875 62396.03125
12 60097.178125 61062.4125
13 58772.1957031 59721.475
14 57481.2488281 58393.1375
15 56182.9453125 57116.06875
16 54920.9054688 55780.44375
17 53665.6953125 54584.65625
18 52446.2683594 53304.93125
19 51279.4480469 52115.528125
20 50121.7476563 50964.05
21 49000.9628906 49723.309375
22 47798.825 48749.8125
23 46726.5738281 47735.9375
24 45666.2964844 46470.38125
25 44583.2070312 45357.409375
26 43544.0882812 44311.9375
27 42484.834375 43322.83125
28 41501.6863281 42317.221875
29 40500.3988281 41225.55
30 39571.2109375 40299.64375
31 38613.7683594 39375.36875
32 37693.6042969 38478.125
33 36765.8863281 37603.88125
34 35847.1931641 36563.86875
35 34998.9105469 35800.725
36 3

In [101]:
with tf.Session() as sess:
    saver.restore(sess, "./model.ckpt")
    for i in range(-N_images):
        x_test = input_metrics_test[i, :, :, :].reshape((1, xdim, ydim, in_chan))
        y_test = output_metric_test[i, :, :, :].reshape((1, xdim, ydim, out_chan))
        
        [output_data] = sess.run([out], feed_dict={X: x_test, Y: y_test});
        
        generate_rainfall_images(output_data, 'rainfall' + str(i) + '_generated.png')
        generate_rainfall_images(y_test, 'rainfall' + str(i) + '_true.png')

INFO:tensorflow:Restoring parameters from ./model.ckpt
